In [1]:
import json
import malaya
from malaya.dictionary import *
from tqdm import tqdm

In [2]:
malay_words = MALAY_WORDS | CAMBRIDGE_MALAY_WORDS | KAMUS_DEWAN_WORDS | DBP_WORDS
malay_words |= negeri | city | country | daerah | parlimen | adun
len(malay_words)

65823

In [3]:
english_words = ENGLISH_WORDS
len(english_words)

234177

In [4]:
english_minus = english_words - malay_words
len(english_minus)

231316

In [5]:
english_minus = {i for i in english_minus if 'haha' not in i and i != 'rt' and 'yeay' not in i and\
                'yes' not in i and 'ooo' not in i and 'insha' not in i and 'huhu' not in i and\
                'insya' not in i and 'hew' not in i and 'uwuu' not in i and\
                 'wkwk' not in i and 'hoho' not in i and 'babi' not in i and\
                 'meow' not in i and 'aiii' not in i and 'alham' not in i and 'mashaa' not in i\
                 and i not in ['takda', 'cer']}
len(english_minus)

231117

In [6]:
import re

def language_detection_textcleaning(string):
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )

    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
    string = string.replace('\n', '').replace('\t', '')

    string = re.sub(
        '[0-9!@#$%^&*()_\\-+{}|\\~`\'";:?/.>,<]', ' ', string, flags=re.UNICODE
    )
    string = re.sub(r'[ ]+', ' ', string).strip()

    return string.lower()


In [7]:
def split(data):
    t = language_detection_textcleaning(data)
    splitted = t.split()
    ngs = set(splitted)
    return ngs

In [8]:
!head -n 100 filter-malay-rojak-rojak.jsonl

In [9]:
# !pip3.8 install lingua-language-detector
# from lingua import Language, LanguageDetectorBuilder
# languages = [Language.ENGLISH, Language.CHINESE, Language.MALAY]
# detector = LanguageDetectorBuilder.from_languages(*languages).build()

In [20]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-v1')

In [21]:
language_detection.predict(['share kalau nak tgk k view wea'])

['rojak']

In [25]:
ms_open = open('filter-malay-rojak-ms.jsonl', 'w')
en_open = open('filter-malay-rojak-en.jsonl', 'w')
rojak_open = open('filter-malay-rojak-rojak.jsonl', 'w')

In [26]:
from tqdm import tqdm

with open('prepare-malay-text-ms.jsonl') as fopen:
    for l in tqdm(fopen):
        
        data = json.loads(l)
        if language_detection.predict([data])[0] not in {'rojak', 'malay'}:
            continue
        t = language_detection_textcleaning(data)
        if not len(t):
            continue
            
        if False:
            f = en_open
        else:
            splitted = t.split()
            ngs = set(splitted)
            len_en = len(ngs & english_minus)
            if len_en == len(ngs):
                f = en_open
            elif len_en:
                f = rojak_open
            else:
                f = ms_open
        
        f.write(f'{json.dumps(t)}\n')
        f.flush()

13573586it [13:18, 19665.78it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
ms_open.close()
en_open.close()
rojak_open.close()